In [1]:
import pandas as pd
from numpy import array, median, exp, sqrt
from hw import Jamshidian 
from hw import hw_helper
from hw import calibration as hw_calib
from hw.const import *
from fox_toolbox.utils import xml_parser, rates
from random import choice
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 5.)})
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2

### Read IRSM FORM

In [2]:
main_curve, sprds = xml_parser.get_rate_curves(INPUT_5SWO)
dsc_curve = main_curve

try:
    estim_curve = sprds[0]
except TypeError:
    estim_curve = main_curve

cal_basket = list(xml_parser.get_calib_basket(INPUT_5SWO))

### READ IRSM OUT

In [3]:
_, irsmout = xml_parser.get_xml(OUTPUT_5SWO)
ref_swos = list(xml_parser.get_calib_basket(irsmout))

ref_mr, (hw_buckets, hw_sigma) = xml_parser.get_hw_params(irsmout)

ref_sigmas = rates.Curve(hw_buckets, hw_sigma, 'PieceWise')

### Jamshidian pricer with ref sigma (Hernard)

In [9]:
calib_premiumsJ = []
debug_df = pd.DataFrame()

swo = cal_basket[2]
      
jamsh_price, debug = Jamshidian.hw_swo(swo, ref_mr, ref_sigmas, dsc_curve, estim_curve)
debug_df = pd.concat([debug_df, pd.DataFrame(data=debug)], sort=False)
calib_premiumsJ.append(jamsh_price)


a = ref_mr
sigma = ref_sigmas

IsCall = False if swo.pay_rec == 'Receiver' else True

coef = Jamshidian.get_coef(swo, a, sigma, dsc_curve, estim_curve)
b_i = Jamshidian.get_b_i(swo, a)
varx = Jamshidian.get_var_x(swo.expiry, a, sigma)


sgn_changes = hw_helper.sign_changes(coef)

x_star = Jamshidian.get_x_star(coef, b_i, varx)

calib_premiumsJ[0], Jamshidian.hw_swo_analytic(coef, b_i, varx, x_star, IsCall)

(0.015152958395710736, 0.015152958395710736)

In [ ]:
(0.015277326631131571, 0.015277326631131571)